In [8]:
# !python --version

In [9]:
# !pip install stable-baselines3[extra]

In [10]:
# !pip install gym
import os
import gym
from stable_baselines3 import PPO #algorithm for agent to learn we dont create neural custom neural network
from stable_baselines3.common.vec_env import DummyVecEnv


from stable_baselines3.common.evaluation import evaluate_policy # to test how well our agent or model performed after training


## 1.Load Environment

In [11]:
env = gym.make("CartPole-v1",render_mode="human") #cart-pole environment

In [12]:
# env.reset() #reset the environment or initilize it to start state
# env.step(1) # to see what our agent get in very step it takes(it returns 5 values 1 array,1 action,2 trunctaed or terminated,1 information total 5)

In [13]:
# env.close()

# Run the below cell to see how cartpole work on random actions

In [14]:
episodes=5
for episode in range(1,episodes+1):
    state=env.reset()
    done= False
    score=0
    while not done:
        action = env.action_space.sample()
        next_state, reward, done, _, _ = env.step(action)
        score+=reward
    
    print("Episode:{} Score:{}".format(episode,score))
# env.close() #to close the rendered frame

     

Episode:1 Score:21.0
Episode:2 Score:14.0
Episode:3 Score:18.0
Episode:4 Score:14.0
Episode:5 Score:24.0


# 3.Train an RL Model

In [1]:
#create folder to store model and log for tensorboard
import os
log_path=os.path.join('pytorch_RL','logs')

In [2]:
log_path


'pytorch_RL/logs'

In [3]:
# PPO is the algorithm we using for this purticular environment



from stable_baselines3 import PPO #algorithm for agent to learn we dont create neural custom neural network
from stable_baselines3.common.vec_env import DummyVecEnv


from stable_baselines3.common.evaluation import evaluate_policy # to test how well our agent or model performed after training
import gymnasium as gym
env = gym.make("CartPole-v1") #cart-pole environment
env=DummyVecEnv([lambda:env]) #act as a wrapper
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path) #seting up the agent using multilayerpAgency

Using cpu device


In [4]:
# PPO??

In [5]:
# lets train the model or agent
model.learn(total_timesteps=20000)

Logging to pytorch_RL/logs/PPO_6
-----------------------------
| time/              |      |
|    fps             | 2074 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1405        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008353247 |
|    clip_fraction        | 0.0972      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00365    |
|    learning_rate        | 0.0003      |
|    loss                 | 8.02        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0166     |
|    value_loss           | 56.3        |
-----------------------------------------
-

# 4. Lets Save and Reload the model

In [6]:
PPO_path=os.path.join('pytorch_RL','saved_model','PPO_model_cartpole')

In [7]:
PPO_path

'pytorch_RL/saved_model/PPO_model_cartpole'

In [8]:
model.save(PPO_path)

We saved the model lets retsore the model again 

In [9]:
del model # delete the model

# now load the model again

In [10]:
model=PPO.load(PPO_path,env=env)

In [11]:
# lets train the restored model
model.learn(total_timesteps=100)

Logging to pytorch_RL/logs/PPO_7
-----------------------------
| time/              |      |
|    fps             | 2132 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------


# 5.Lets do the evaluation by testing 

# so far we not used the trained model...Lets do that!!!

In [12]:
env = gym.make("CartPole-v1") #cart-pole environment
env=DummyVecEnv([lambda:env]) #act as a wrapper

evaluate_policy(model,env,n_eval_episodes=10) #evaluate our model and visulaize at the same time for 10 sec
# evaluate_policy??


/home/noushad/.conda/envs/pytorch/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(441.7, 59.42566785489247)

In [13]:
env.close() # to close the frame

# 6.Lets Test the model now

In [16]:
obs=env.reset()

In [17]:
action,_=model.predict(obs)

In [18]:
env.step(action)

(array([[-0.01806887, -0.21850213, -0.03100273,  0.30269402]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{'TimeLimit.truncated': False}])

In [ ]:
episodes=5
for episode in range(1,episodes+1):
    obs=env.reset()
    done= False
    score=0
    while not done:
        action,_ = model.predict(obs) #instead of taking random action our model predict the best action for current state
        next_state, reward, done, _, _ = env.step(action)
        score+=reward
    
    print("Episode:{} Score:{}".format(episode,score))
# env.close() #to close the rendered frame

     

# 7. Viewing logs in the Tensorboard

In [22]:
# lets define the path where the logs are stored
training_log_path=os.path.join(log_path,'PPO_1')
training_log_path

'pytorch_RL/logs/PPO_1'

In [26]:
!tensorboard --logdir={training_log_path}

TensorFlow installation not found - running with reduced feature set.
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.16.2 at http://localhost:6020/ (Press CTRL+C to quit)
^C


# 8. Adding a callback to the training stage

In [28]:
from stable_baselines3.common.callbacks import EvalCallback,StopTrainingOnRewardThreshold

In [33]:
save_path=os.path.join('pytorch_RL','saved_model')

In [37]:
stop_callback=StopTrainingOnRewardThreshold(reward_threshold=200,verbose=1)
eval_callback=EvalCallback(env,callback_on_new_best=stop_callback,
                           eval_freq=10000,
                           best_model_save_path=save_path,verbose=1)

# EvalCallback??

In [38]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path) #seting up the agent using multilayerpAgency

Using cpu device


In [39]:
model.learn(total_timesteps=20000,callback=eval_callback) #it save every 10000 steps

Logging to pytorch_RL/logs/PPO_8
-----------------------------
| time/              |      |
|    fps             | 2114 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1422         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0075712726 |
|    clip_fraction        | 0.091        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | 0.000642     |
|    learning_rate        | 0.0003       |
|    loss                 | 6.47         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0138      |
|    value_loss           | 48.6         |
--------------------------

# 8. Lets use a different custom neural network for training (different policy)(its all done using stable documentation)

In [45]:
# vf=value function # pi 4 layers with 128  neurons or weights
net_arch=[dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [46]:
model=PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path, policy_kwargs={'net_arch':net_arch}) # we addes our custom network to the algorithm

Using cpu device


In [47]:
model.learn(total_timesteps=20000,callback=eval_callback) #it save every 10000 steps

Logging to pytorch_RL/logs/PPO_9
-----------------------------
| time/              |      |
|    fps             | 1881 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1096        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014420839 |
|    clip_fraction        | 0.213       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | 0.000723    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.21        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.025      |
|    value_loss           | 20.6        |
-----------------------------------------
-

# 9. Finally use an alternate algoirthm instead of PPO

In [50]:
#Lets use DQN instead of PPM
from stable_baselines3 import DQN

In [51]:
model=DQN('MlpPolicy',env,verbose=1,tensorboard_log=log_path) #replaced PPO

Using cpu device


In [52]:
model.learn(total_timesteps=20000)

Logging to pytorch_RL/logs/DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.956    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 6887     |
|    time_elapsed     | 0        |
|    total_timesteps  | 93       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.922    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2802     |
|    time_elapsed     | 0        |
|    total_timesteps  | 164      |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 0.624    |
|    n_updates        | 15       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.886    |
| time/               |          |
|    episodes         | 12       |
|    fps              